In [ ]:
#!pip install transformers datasets peft loralib accelerate
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset
from peft import LoraConfig, get_peft_model

In [ ]:
# Step 1: Load Model and Tokenizer
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Add padding token if not exists (important for some models)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
# Step 2: Apply LoRA Config
lora_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=["c_attn"],         # Correct module name for distilgpt2
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,179,648 || all params: 83,092,224 || trainable%: 1.4197


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
# Step 3: Load and Prepare Dataset
dataset = load_dataset("json", data_files="extensive_excuse_dataset.jsonl")

def tokenize_function(examples):
    combined = [p + "\n" + e for p, e in zip(examples["prompt"], examples["excuse"])]
    tokenized = tokenizer(
        combined,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_special_tokens_mask=True,
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(tokenize_function, batched=True, batch_size=32)
tokenized_dataset = tokenized_dataset.remove_columns(["prompt", "excuse"])

# Split into train/val
train_val_split = tokenized_dataset["train"].train_test_split(test_size=0.1, seed=42)
train_dataset = train_val_split["train"]
eval_dataset = train_val_split["test"]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# Step 4: Define Training Arguments
training_args = TrainingArguments(
    output_dir="./distilgpt2-excuse-lora",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
    optim="adamw_torch",
    learning_rate=2e-4,
    warmup_steps=100,
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

In [ ]:
# Step 5: Create Trainer and Train
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

print("Starting training...")
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Starting training...


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,0.544300,0.533065
2,0.469100,0.451638
3,0.418800,0.421721
4,0.444100,0.406167
5,0.416300,0.400185


TrainOutput(global_step=1125, training_loss=1.0788609040578205, metrics={'train_runtime': 4588.9383, 'train_samples_per_second': 0.981, 'train_steps_per_second': 0.245, 'total_flos': 151056285696000.0, 'train_loss': 1.0788609040578205, 'epoch': 5.0})

In [ ]:
# -------------------------------
# Step 6: Save Only LoRA Adapter Weights
# -------------------------------
model.save_pretrained("./distilgpt2-excuse-lora_adapter")
tokenizer.save_pretrained("./distilgpt2-excuse-lora_adapter")

print("✅ LoRA adapter saved to './distilgpt2-excuse-lora_adapter'")

✅ LoRA adapter saved to './distilgpt2-excuse-lora_adapter'


# **Use the Trained Adapter Later**

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Load base model
base_model = AutoModelForCausalLM.from_pretrained("distilgpt2")
tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

# Load adapter
model = PeftModel.from_pretrained(base_model, "./distilgpt2-excuse-lora_adapter")

# Generate an excuse
prompt = "Why didn't you finish the assignment?"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=100, do_sample=True)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Why didn't you finish the assignment?
I couldn't do everything but focus.


**For better generation quality, consider using prompt formatting like**

```
Prompt: [your prompt here]
Excuse:
```